In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import AnalysisFunctions as afuns
from functools import reduce
import os
import cv2

In [2]:
root_path = r'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')

In [3]:
def get_bdata_filename(exp_path):
    bdata_filename = [filename for filename in os.listdir(exp_path+sep+r'with food\blob analysis normalized by white paper') 
                        if filename.startswith(r'bdata')][0]
    return bdata_filename

In [4]:
exp_num = 19
exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)
bdata_filename = get_bdata_filename(exp_path)

bdata = pd.read_csv(exp_path + sep + r'with food\blob analysis normalized by white paper' + sep + bdata_filename, index_col=0)
fdata = pd.read_csv(exp_path + sep + r'forager_table_with_feeding_sizes_ul_transparency_corrected.csv', index_col=[0])
tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
crops = pd.read_csv(exp_path + sep + r'clean_crops_transparency_corrected.csv', header=[0,1], index_col=[0]).swaplevel(axis=1)
ants = pd.read_csv(exp_path + sep + r'ants_list.csv')
conversion_factors = pd.read_csv(exp_path + sep + r'conversion_factors_by_weight_and_feeding_sum.csv')
transparency = pd.read_csv(exp_path + sep + r'transparency_table.csv')

In [5]:
def get_ant_data(ant, bdata, crops, conversion_factors=False): #tdata, transparency, dt=10, fdata=False,
    ant_data={}
    
    ant_data['ant'] = ant
    
    raw_x = bdata['a'+str(ant)+'-x']
    raw_y = bdata['a'+str(ant)+'-y']
    ant_data['x']=raw_x.interpolate(method='linear',limit_area='inside')
    ant_data['y']=raw_y.interpolate(method='linear',limit_area='inside')
    
    ant_data['red_crop'] = crops[('red',str(ant))]/conversion_factors['red'][0]
    ant_data['yellow_crop'] = crops[('yellow',str(ant))]/conversion_factors['yellow'][0]
    
#     ant_data['interactions']=tdata[(tdata['giver']==ant) | (tdata['receiver']==ant)]
    
#     if isinstance(fdata, pd.DataFrame):
#         ant_data['feedings'] = fdata[fdata.ant_id==ant]
    
    return ant_data

In [ ]:
def center_of_mass(x_list, y_list, mass_list):
    
    x_center = np.ma.average(x_list,weights=mass_list)
    y_center = np.ma.average(y_list,weights=mass_list)
    return [x_center, y_center]

In [ ]:
def get_frame_data(ants,frame_idx):
    frame_df = pd.DataFrame(columns=['ant','x','y','red_crop','yellow_crop'])
    ant_col=[]
    x_col=[]
    y_col=[]
    red_crop_col=[]
    yellow_crop_col=[]
    for ant in ants:
        ant_row = pd
        ant_col.append(ant['ant'])
        x_col.append(ant['x'][frame_idx])
        y_col.append(ant['y'][frame_idx])
        red_crop_col.append(ant['red_crop'][frame_idx])
        yellow_crop_col.append(ant['yellow_crop'][frame_idx])
    frame_df['ant']=ant_col
    frame_df['x']=x_col
    frame_df['y']=y_col
    frame_df['red_crop']=red_crop_col
    frame_df['yellow_crop']=yellow_crop_col
    return frame_df

In [ ]:
vid = cv2.VideoCapture(exp_path + sep + r'with food\blob analysis normalized by white paper' + sep + r"analyzed movie.avi")
ret, img = vid.read()
scale_percent = 10 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

plt.figure()
cv2.imshow('',resized)

In [ ]:
def get_nest_wall_x():
    # Todo
    return nest_wall_x

In [ ]:
all_ants_data=[]
for ant in ants['ant_id']:
    all_ants_data.append(get_ant_data(ant, bdata, crops, conversion_factors))

In [ ]:
nest_wall_x=3100
red_centers=[]
yellow_centers=[]
for frame in np.arange(7500,8500,2):
    frame_df = get_frame_data(all_ants_data,frame)
    frame_df=frame_df.dropna()
    frame_df=frame_df[frame_df['x']<=nest_wall_x]
    red_centers.append(center_of_mass(frame_df.dropna()['x'],frame_df['y'],frame_df['red_crop']))
    yellow_centers.append(center_of_mass(frame_df.dropna()['x'],frame_df['y'],frame_df['yellow_crop']))


In [ ]:
plt.plot(np.array(red_centers).transpose()[0],np.array(red_centers).transpose()[1],'.r')
plt.plot(np.array(yellow_centers).transpose()[0],np.array(yellow_centers).transpose()[1],'.y')

In [ ]:
plt.scatter(frame_df['x'],frame_df['y'],s=50*frame_df['red_crop'],color='r',alpha=0.5)
plt.scatter(frame_df['x'],frame_df['y'],s=50*frame_df['yellow_crop'],color='y',alpha=0.5)
plt.plot(red_centers[-1][0],red_centers[-1][1],'xk',markersize=20)
plt.plot(yellow_centers[-1][0],yellow_centers[-1][1],'^k',markersize=20)

In [ ]:
center_of_mass(frame_df.dropna()['x'],frame_df['y'],frame_df['red_crop'])

In [ ]:
a=[]
a.append(center_of_mass(frame_df.dropna()['x'],frame_df['y'],frame_df['red_crop']))
a

In [ ]:
a.append(center_of_mass(frame_df.dropna()['x'],frame_df['y'],frame_df['red_crop']))
a

In [ ]:
np.array(red_centers).transpose()[0]